In [3]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LogisticRegression

In [4]:
train = pd.read_csv("/kaggle/input/hms-harmful-brain-activity-classification/train.csv")
train

,eeg_id,eeg_sub_id,eeg_label_offset_seconds,spectrogram_id,spectrogram_sub_id,spectrogram_label_offset_seconds,label_id,patient_id,expert_consensus,seizure_vote,lpd_vote,gpd_vote,lrda_vote,grda_vote,other_vote
0,1628180742,0,0.0,353733,0,0.0,127492639,42516,Seizure,3,0,0,0,0,0
1,1628180742,1,6.0,353733,1,6.0,3887563113,42516,Seizure,3,0,0,0,0,0
2,1628180742,2,8.0,353733,2,8.0,1142670488,42516,Seizure,3,0,0,0,0,0
3,1628180742,3,18.0,353733,3,18.0,2718991173,42516,Seizure,3,0,0,0,0,0
4,1628180742,4,24.0,353733,4,24.0,3080632009,42516,Seizure,3,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
106795,351917269,6,12.0,2147388374,6,12.0,4195677307,10351,LRDA,0,0,0,3,0,0
106796,351917269,7,14.0,2147388374,7,14.0,290896675,10351,LRDA,0,0,0,3,0,0
106797,351917269,8,16.0,2147388374,8,16.0,461435451,10351,LRDA,0,0,0,3,0,0
106798,351917269,9,18.0,2147388374,9,18.0,3786213131,10351,LRDA,0,0,0,3,0,0


In [5]:
vote_cols = [col for col in train if col.endswith("_vote")]

train['sum_votes'] = sum([train[col] for col in vote_cols])

for col in vote_cols:
    train[col] = train[col] / train['sum_votes']
    
train

,eeg_id,eeg_sub_id,eeg_label_offset_seconds,spectrogram_id,spectrogram_sub_id,spectrogram_label_offset_seconds,label_id,patient_id,expert_consensus,seizure_vote,lpd_vote,gpd_vote,lrda_vote,grda_vote,other_vote,sum_votes
0,1628180742,0,0.0,353733,0,0.0,127492639,42516,Seizure,1.0,0.0,0.0,0.0,0.0,0.0,3
1,1628180742,1,6.0,353733,1,6.0,3887563113,42516,Seizure,1.0,0.0,0.0,0.0,0.0,0.0,3
2,1628180742,2,8.0,353733,2,8.0,1142670488,42516,Seizure,1.0,0.0,0.0,0.0,0.0,0.0,3
3,1628180742,3,18.0,353733,3,18.0,2718991173,42516,Seizure,1.0,0.0,0.0,0.0,0.0,0.0,3
4,1628180742,4,24.0,353733,4,24.0,3080632009,42516,Seizure,1.0,0.0,0.0,0.0,0.0,0.0,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
106795,351917269,6,12.0,2147388374,6,12.0,4195677307,10351,LRDA,0.0,0.0,0.0,1.0,0.0,0.0,3
106796,351917269,7,14.0,2147388374,7,14.0,290896675,10351,LRDA,0.0,0.0,0.0,1.0,0.0,0.0,3
106797,351917269,8,16.0,2147388374,8,16.0,461435451,10351,LRDA,0.0,0.0,0.0,1.0,0.0,0.0,3
106798,351917269,9,18.0,2147388374,9,18.0,3786213131,10351,LRDA,0.0,0.0,0.0,1.0,0.0,0.0,3


In [6]:
X_train = []

for eeg_id in train.eeg_id.unique():
    eeg = pd.read_parquet(f"/kaggle/input/hms-harmful-brain-activity-classification/train_eegs/{eeg_id}.parquet")
    X_train.append(eeg.to_numpy(na_value=0)[:1000,:].reshape(1000*eeg.shape[1]))

X_train = np.array(X_train)

In [7]:
Y_train = train.drop_duplicates(['eeg_id'], keep='first')[vote_cols]
Y_train = Y_train.idxmax(axis=1)

In [8]:
clf = LogisticRegression().fit(X_train, Y_train)

/opt/conda/lib/python3.10/site-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [9]:
test = pd.read_csv("/kaggle/input/hms-harmful-brain-activity-classification/test.csv")
test

,spectrogram_id,eeg_id,patient_id
0,853520,3911565283,6885


In [10]:
X_test = []

for eeg_id in test.eeg_id.unique():
    eeg = pd.read_parquet(f"/kaggle/input/hms-harmful-brain-activity-classification/test_eegs/{eeg_id}.parquet")
    X_test.append(eeg.to_numpy(na_value=0)[:1000,:].reshape(1000*eeg.shape[1]))

X_test = np.array(X_test)

In [11]:
np.exp(clf.predict_log_proba(X_test))

array([[0.16284082, 0.17401217, 0.15114491, 0.16887565, 0.1520956 ,
        0.19103084]])

In [19]:
result = pd.concat([test['eeg_id'], pd.DataFrame(np.exp(clf.predict_log_proba(X_test)), columns=clf.classes_)], axis=1)
result

,eeg_id,gpd_vote,grda_vote,lpd_vote,lrda_vote,other_vote,seizure_vote
0,3911565283,0.162841,0.174012,0.151145,0.168876,0.152096,0.191031


In [20]:
result.to_csv("submission.csv")